In [1]:
#export
from k1lib.callbacks import Callback, Callbacks
import k1lib

In [2]:
#export
@k1lib.patch(Callback.cls)
class BatchLimit(Callback):
    """Cancels the epoch after executed certain number of batches"""
    def __init__(self, limit):
        super().__init__()
        self.limit = limit if limit != None else float("inf")
    def startEpoch(self): self.currentBatch = 0
    def startBatch(self):
        if self.currentBatch >= self.limit:
            raise k1lib.CancelEpochException()
    def endBatch(self): self.currentBatch += 1
@k1lib.patch(Callbacks, docs=BatchLimit)
def withBatchLimit(self, limit, name=None): return self.append(BatchLimit(limit), name)

In [3]:
#export
@k1lib.patch(Callback.cls)
class EpochLimit(Callback):
    """Cancels the run after executed certain number of epochs"""
    def __init__(self, limit):
        super().__init__()
        self.limit = limit if limit != None else float("inf")
    def startRun(self): self.currentEpoch = 0
    def startEpoch(self):
        if self.currentEpoch >= self.limit:
            raise k1lib.CancelRunException()
    def endEpoch(self): self.currentEpoch += 1
@k1lib.patch(Callbacks, docs=EpochLimit)
def withEpochLimit(self, limit, name=None): return self.append(EpochLimit(limit), name)

In [4]:
#export
@k1lib.patch(Callback.cls)
class CancelOnExplosion(Callback):
    """Cancels the run if any of the parameters are larger than a certain limit"""
    def __init__(self, limit=1e6):
        super().__init__()
        self.order = 18; self.limit = limit; self.triggered = False
    def startRun(self): self.triggered = False
    def startBatch(self):
        for p in self.model.parameters():
            o = p.detach()
            if o.max() > self.limit or o.min() < -self.limit:
                self.triggered = True
                raise k1lib.CancelRunException("Explosion detected!")
    def __repr__(self):
        return f"""{self._reprHead}, use...
- cb.triggered: to see if there was an explosion on the last run
- cb.progress: to see current progress at explosion time
{self._reprCan}"""
@k1lib.patch(Callbacks, docs=CancelOnExplosion)
def withCancelOnExplosion(self, limit=1e6):
    return self.append(CancelOnExplosion(limit))

In [5]:
#export
@k1lib.patch(Callback.cls)
class CancelOnLowLoss(Callback):
    """Cancels the run if loss is lower than amount specified.
Args:
    loss (float): <obv>
    epochMode (bool): false if use batch loss, true if use valid epoch loss"""
    def __init__(self, loss, epochMode=False):
        super().__init__(); self.loss = loss; self.epochMode = epochMode
    def endBatch(self):
        if self.epochMode:
            if not hasattr(self, "Loss"): return
            data = self.Loss.epoch.valid
            if len(data) > 0 and data[-1] < self.loss:
                raise k1lib.CancelRunException(f"Low loss {self.loss} achieved!")
        elif self.learner.loss < self.loss:
            raise k1lib.CancelRunException(f"Low loss {self.loss} achieved!")
@k1lib.patch(Callbacks, docs=CancelOnLowLoss)
def withCancelOnLowLoss(self, loss, epochMode=False):
    return self.append(CancelOnLowLoss(loss, epochMode))

In [6]:
#export
@k1lib.patch(Callback.cls)
class DontTrain(Callback):
    def startBackward(self): return True
    def startStep(self): return True
@k1lib.patch(Callbacks)
def withDontTrain(self, name=None): return self.append(DontTrain(), name)

In [7]:
!../../export.py _callbacks/limits

Current dir: /home/kelvin/repos/labs/k1lib, ../../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 0.1.0
Uninstalling k1lib-0.1.0:
  Successfully uninstalled k1lib-0.1.0
running install
running bdist_egg
running egg_info
creating k1lib.egg-info
writing k1lib.egg-info/PKG-INFO
writing dependency_links to k1lib.egg-info/dependency_links.txt
writing top-level names to k1lib.egg-info/top_level.txt
writing manifest file 'k1lib.egg-info/SOURCES.txt'
reading manifest file 'k1lib.egg-info/SOURCES.txt'
writing manifest file 'k1lib.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/k1lib
copying k1lib/_learner.py -> build/lib/k1lib
copying k1lib/callbacks.py -> build/lib/k1lib
copying k1lib/data.py -> build/lib/k1lib
copying k1lib/imports.py -> build/lib/k1lib
copying k1lib/_basics.py -> build/lib/k1lib
copying k1lib/nn.py -> 